In [4]:
import os
import time

import numpy
from PIL import Image

image_dataset = []

#Create Random Image functionality credited to https://gist.github.com/sparkstar/c6bff470e84912566113
def create_image(width = 1920, height = 1080, num_of_images = 100):
    width = int(width)
    height = int(height)
    num_of_images = int(num_of_images)
 
    #current = time.strftime("%Y%m%d%H%M%S")
    #os.mkdir(current)
 
    for n in range(num_of_images):
        #filename = '{0}/{0}_{1:03d}.jpg'.format(current, n)
        rgb_array = numpy.random.rand(height,width,3) * 255
        image = Image.fromarray(rgb_array.astype('uint8')).convert('RGB')
        #image.save(filename)
        image_dataset.append(image)

#Specify image dimensions
image_dimensions=input("Specify image dimensions ")
#Specify each pixel's permutation
pixel_mutations = []
pixel_mutation = ""
while pixel_mutation != "end":    
    pixel_mutation = input("Specify each pixel's permutation - use X x Y to indicate pixel, followed by +/- values for RGB permutations ")
    pixel_mutations.append(pixel_mutation)
#output data so can be used to create backdoors
output_data = [image_dimensions, pixel_mutations]
print(output_data)


image_dimension_numbers = []
for word in image_dimensions.split():
   if word.isdigit():
      image_dimension_numbers.append(int(word))

print("numbers check: "+str(image_dimension_numbers[0]) + " "+ str(image_dimension_numbers[1]))

create_image(image_dimension_numbers[0], image_dimension_numbers[1], 1)

#load image dataset

print("listing images...")
for image in image_dataset:
    display(image)

    #poison the data

    for individual_pixel_mutation in pixel_mutations:
        individual_pixel_mutation_numbers = []
        for word_two in individual_pixel_mutation.split():
            if word_two.lstrip('-').isdigit():
                individual_pixel_mutation_numbers.append(int(word_two))
        print(individual_pixel_mutation_numbers)
        if individual_pixel_mutation_numbers:
            print("individual_pixel_mutation_numbers check: "+str(individual_pixel_mutation_numbers[0])+ " "+str(individual_pixel_mutation_numbers[1])+ " "+str(individual_pixel_mutation_numbers[2])+ " "+str(individual_pixel_mutation_numbers[3])+ " "+str(individual_pixel_mutation_numbers[4]))
            #pinpoint the exact pixel and add the permutations
            pixelmap = image.load()
            individual_pixel = pixelmap[individual_pixel_mutation_numbers[0]-1, individual_pixel_mutation_numbers[1]-1]
            print("Confirming pixel check: "+str(individual_pixel[0])+ " "+str(individual_pixel[1])+ " "+str(individual_pixel[2]))
            R_value = -1
            G_value = -1
            B_value = -1
            
            #Absolute pixel value mutation - setting mutation absolutely
            if (individual_pixel_mutation_numbers[5] == 1):
                if(individual_pixel_mutation_numbers[2]>=0):
                    R_value = individual_pixel_mutation_numbers[2]
                if(individual_pixel_mutation_numbers[3]>=0):
                    G_value = individual_pixel_mutation_numbers[3]
                if(individual_pixel_mutation_numbers[4]>=0):
                    B_value = individual_pixel_mutation_numbers[4]
                if(R_value == -1 or G_value == -1 or B_value == -1):
                    print("Error: Cannot compute absolute negative pixel values. Setting negative values to zero")
                
            #Relative pixel value mutation - setting mutation relative to existing pixel values
            elif(individual_pixel_mutation_numbers[5] == 0):
                if(individual_pixel[0]+individual_pixel_mutation_numbers[2]>255):
                    R_value = 255
                elif ((individual_pixel[0]+individual_pixel_mutation_numbers[2])<0):
                    R_value = 0
                else: R_value = individual_pixel[0]+individual_pixel_mutation_numbers[2]
    
                if(individual_pixel[1]+individual_pixel_mutation_numbers[3]>255):
                    G_value = 255
                elif ((individual_pixel[1]+individual_pixel_mutation_numbers[3])<0):
                    G_value = 0
                else: G_value = individual_pixel[1]+individual_pixel_mutation_numbers[3]
    
                if(individual_pixel[2]+individual_pixel_mutation_numbers[4]>255):
                    B_value = 255
                elif ((individual_pixel[2]+individual_pixel_mutation_numbers[4])<0):
                    B_value = 0
                else: B_value = individual_pixel[2]+individual_pixel_mutation_numbers[4]
        
            pixelmap[individual_pixel_mutation_numbers[0]-1, individual_pixel_mutation_numbers[1]-1] = (R_value, G_value, B_value)
            mutated_individual_pixel = pixelmap[individual_pixel_mutation_numbers[0]-1, individual_pixel_mutation_numbers[1]-1]
            print("Confirming mutated pixel check: "+str(mutated_individual_pixel[0])+ " "+str(mutated_individual_pixel[1])+ " "+str(mutated_individual_pixel[2]))
    display(image)

Specify image dimensions 2 x 2
Specify each pixel's permutation - use X x Y to indicate pixel, followed by +/- values for RGB permutations 1 x 2 100 200 14 1
Specify each pixel's permutation - use X x Y to indicate pixel, followed by +/- values for RGB permutations end
['2 x 2', ['1 x 2 100 200 14 1', 'end']]
numbers check: 2 2
listing images...


[1, 2, 100, 200, 14, 1]
individual_pixel_mutation_numbers check: 1 2 100 200 14
Confirming pixel check: 157 46 156
Confirming mutated pixel check: 100 200 14
[]
